In [41]:
import polars as pl

In [42]:
# 1.
# Les inn kjoretoyinfo_fra_[putt_inn_år].parquet. Bruk pl.scan_parquet.

pl_df = pl.scan_parquet("filer/kjoretoyinfo_fra_2000.parquet").collect()

pl_df

tekn_reg_f_g,tekn_reg_f_g_n,tekn_reg_eier_dato,tekn_merke,tekn_modell,tekn_farge,tekn_drivstoff,tekn_siste_pkk,tekn_neste_pkk,tekn_aksler_drift,tekn_sitteplasser_totalt
i64,i64,i64,i64,str,str,str,str,str,i64,i64
20211013,20211013,20211013,3900,"""VITO""","""05""","""2""",null,"""20251013""",2,3
20200129,20200129,20200129,5750,"""CADDY""","""06""","""2""","""20231205""","""20260129""",2,2
20231129,20231129,20231129,2600,"""Mustang Mach-E…","""06""","""5""",null,"""20271129""",1,5
20200127,20200127,20200127,5750,"""CADDY""","""01""","""2""",null,"""20240127""",2,2
20180731,20190130,20190130,2600,"""F-150 Platinum…","""06""","""2""","""20220503""","""20240503""",2,2
20200519,20200519,20200519,5750,"""CRAFTER""","""01""","""2""",null,"""20240519""",1,3
20200911,20200911,20220216,5150,"""CITIGO""","""01""","""5""",null,"""20240911""",1,4
20220512,20220512,20220512,5480,"""YARIS""","""04""","""1,5""",null,"""20260512""",1,5
20221104,20221104,20221104,5480,"""RAV4""","""06""","""1,5,5""",null,"""20261104""",2,5


In [43]:
# 2. 
# “tekn_reg_f_g_n” er dato for førstegangsregistrering i Norge. 
# Konverter denne kolonnen til en datetime.
# Gjør tilsvarende for “neste_pkk” og “tekn_reg_eier_dato”.

pl_df = pl_df.with_columns(
    tekn_reg_f_g_n = pl.col("tekn_reg_f_g_n").cast(str).str.to_date(format="%Y%m%d"),
    tekn_neste_pkk = pl.when(
        pl.col("tekn_neste_pkk").str.len_bytes() == 8
    ).then(
        pl.col("tekn_neste_pkk").cast(str).str.to_date(format="%Y%m%d", strict=False)
    ).otherwise(
        pl.lit(None)
    ),
    tekn_reg_eier_dato = pl.col("tekn_reg_eier_dato").cast(str).str.to_date(format="%Y%m%d")
)

pl_df

tekn_reg_f_g,tekn_reg_f_g_n,tekn_reg_eier_dato,tekn_merke,tekn_modell,tekn_farge,tekn_drivstoff,tekn_siste_pkk,tekn_neste_pkk,tekn_aksler_drift,tekn_sitteplasser_totalt
i64,date,date,i64,str,str,str,str,date,i64,i64
20211013,2021-10-13,2021-10-13,3900,"""VITO""","""05""","""2""",null,2025-10-13,2,3
20200129,2020-01-29,2020-01-29,5750,"""CADDY""","""06""","""2""","""20231205""",2026-01-29,2,2
20231129,2023-11-29,2023-11-29,2600,"""Mustang Mach-E…","""06""","""5""",null,2027-11-29,1,5
20200127,2020-01-27,2020-01-27,5750,"""CADDY""","""01""","""2""",null,2024-01-27,2,2
20180731,2019-01-30,2019-01-30,2600,"""F-150 Platinum…","""06""","""2""","""20220503""",2024-05-03,2,2
20200519,2020-05-19,2020-05-19,5750,"""CRAFTER""","""01""","""2""",null,2024-05-19,1,3
20200911,2020-09-11,2022-02-16,5150,"""CITIGO""","""01""","""5""",null,2024-09-11,1,4
20220512,2022-05-12,2022-05-12,5480,"""YARIS""","""04""","""1,5""",null,2026-05-12,1,5
20221104,2022-11-04,2022-11-04,5480,"""RAV4""","""06""","""1,5,5""",null,2026-11-04,2,5


In [44]:
# 3.
# Join inn navnet på fargen, det vil si, join “tekn_farge” med “kode” i fargekode.csv.
# Pass på at du ikke mister noen biler som vi ikke vet fargen på.

# Manipulrerer datasettet ved å joine det med et annet datasett: 
# Scanner inn og klargjør fargekode.csv for bruk av lazyFrame. Henter rader ved bruke av .collect.
# Truncate_ragged_lines=True = Denne funksjonen er benyttet for at systemet skal ignorere avvik i forventede verdier,
# og faktiske verdier i tuppelen.
fargekode_df = pl.scan_csv("filer/fargekode.csv", separator=";", truncate_ragged_lines=True).collect()

#Fikser verdieen i kolonnen "tekn_farge" ved å fjerne "0",
# slik at den stemmer overens med verdiene kolonnen "kode" i hjelpetabellen "fargekode".
pl_df = pl_df.with_columns(
    pl_df["tekn_farge"].str.strip_chars("0")
)

# Konverter "kode" til streng og gi den et annet navn.
fargekode_df = fargekode_df.select([
    pl.col("*"),
    pl.col("kode").cast(pl.String).alias("fargekode")
])

# Utfør venstre join basert på kolonnene tekn_farge og kode.
joined_df = pl_df.join(
    fargekode_df,
    left_on="tekn_farge",  # Kolonnenavn i pl_df.
    right_on="fargekode",  # Kolonnenavn i fargekode_df.
    how="left"
)

# Forsikrer at vi kan se biler vi ikke vet fargekoden på ved å legge inn en betingelse "When". 
# Hvis betingelsen "when" blir oppfylt genereres en tekstsom forteller bruker at "fargekoden er ukjent" 
# på den akutelle bilen. 
# Hvis fargekoden derimot er tilstede i kolonnen for farge, genereres teksten fra den opprinnelige tabellen.

joined_df = joined_df.with_columns(
    beskrivelse=pl.when(joined_df["kode"].is_null()).then(pl.lit("Ukjent farge")).otherwise(joined_df["beskrivelse"])
)

joined_df

tekn_reg_f_g,tekn_reg_f_g_n,tekn_reg_eier_dato,tekn_merke,tekn_modell,tekn_farge,tekn_drivstoff,tekn_siste_pkk,tekn_neste_pkk,tekn_aksler_drift,tekn_sitteplasser_totalt,kode,beskrivelse
i64,date,date,i64,str,str,str,str,date,i64,i64,i64,str
20211013,2021-10-13,2021-10-13,3900,"""VITO""","""5""","""2""",null,2025-10-13,2,3,5,"""Grå"""
20200129,2020-01-29,2020-01-29,5750,"""CADDY""","""6""","""2""","""20231205""",2026-01-29,2,2,6,"""Svart (også bl…"
20231129,2023-11-29,2023-11-29,2600,"""Mustang Mach-E…","""6""","""5""",null,2027-11-29,1,5,6,"""Svart (også bl…"
20200127,2020-01-27,2020-01-27,5750,"""CADDY""","""1""","""2""",null,2024-01-27,2,2,1,"""Hvit (også ant…"
20180731,2019-01-30,2019-01-30,2600,"""F-150 Platinum…","""6""","""2""","""20220503""",2024-05-03,2,2,6,"""Svart (også bl…"
20200519,2020-05-19,2020-05-19,5750,"""CRAFTER""","""1""","""2""",null,2024-05-19,1,3,1,"""Hvit (også ant…"
20200911,2020-09-11,2022-02-16,5150,"""CITIGO""","""1""","""5""",null,2024-09-11,1,4,1,"""Hvit (også ant…"
20220512,2022-05-12,2022-05-12,5480,"""YARIS""","""4""","""1,5""",null,2026-05-12,1,5,4,"""Grønn (også gr…"
20221104,2022-11-04,2022-11-04,5480,"""RAV4""","""6""","""1,5,5""",null,2026-11-04,2,5,6,"""Svart (også bl…"


In [45]:
# 4. 
# Lag en kolonne “elbil” som er sann hvis drivstofftypen er kun elektrisk, false ellers.

joined_df = joined_df.with_columns(
    elbil=pl.col("tekn_drivstoff").eq(pl.lit("5"))
);

joined_df

tekn_reg_f_g,tekn_reg_f_g_n,tekn_reg_eier_dato,tekn_merke,tekn_modell,tekn_farge,tekn_drivstoff,tekn_siste_pkk,tekn_neste_pkk,tekn_aksler_drift,tekn_sitteplasser_totalt,kode,beskrivelse,elbil
i64,date,date,i64,str,str,str,str,date,i64,i64,i64,str,bool
20211013,2021-10-13,2021-10-13,3900,"""VITO""","""5""","""2""",null,2025-10-13,2,3,5,"""Grå""",false
20200129,2020-01-29,2020-01-29,5750,"""CADDY""","""6""","""2""","""20231205""",2026-01-29,2,2,6,"""Svart (også bl…",false
20231129,2023-11-29,2023-11-29,2600,"""Mustang Mach-E…","""6""","""5""",null,2027-11-29,1,5,6,"""Svart (også bl…",true
20200127,2020-01-27,2020-01-27,5750,"""CADDY""","""1""","""2""",null,2024-01-27,2,2,1,"""Hvit (også ant…",false
20180731,2019-01-30,2019-01-30,2600,"""F-150 Platinum…","""6""","""2""","""20220503""",2024-05-03,2,2,6,"""Svart (også bl…",false
20200519,2020-05-19,2020-05-19,5750,"""CRAFTER""","""1""","""2""",null,2024-05-19,1,3,1,"""Hvit (også ant…",false
20200911,2020-09-11,2022-02-16,5150,"""CITIGO""","""1""","""5""",null,2024-09-11,1,4,1,"""Hvit (også ant…",true
20220512,2022-05-12,2022-05-12,5480,"""YARIS""","""4""","""1,5""",null,2026-05-12,1,5,4,"""Grønn (også gr…",false
20221104,2022-11-04,2022-11-04,5480,"""RAV4""","""6""","""1,5,5""",null,2026-11-04,2,5,6,"""Svart (også bl…",false


In [49]:
# 5. 
# Hent ut bare kolonnene: tekn_reg_f_g_n, tekn_reg_eier_dato, tekn_aksler_drift, tekn_merke, tekn_modell,
# tekn_drivstoff, tekn_neste_pkk, farge på bilen fra fargekode.csv,
# tekn_drivstoff, kolonne som indikerer om bilen er en elbil.

#Lagrer det foretrukne datasettet

pl_df = joined_df.select([
    pl.col("tekn_reg_f_g_n"), 
    pl.col("tekn_reg_eier_dato"), 
    pl.col("tekn_aksler_drift"), 
    pl.col("tekn_merke"), 
    pl.col("tekn_modell"), 
    pl.col("tekn_drivstoff"), 
    pl.col("tekn_neste_pkk"), 
    pl.col("kode").alias("farge_kode"), 
    pl.col("beskrivelse").alias("farge_beskrivelse"),  
    pl.col("elbil")
])

pl_df

tekn_reg_f_g_n,tekn_reg_eier_dato,tekn_aksler_drift,tekn_merke,tekn_modell,tekn_drivstoff,tekn_neste_pkk,farge_kode,farge_beskrivelse,elbil
date,date,i64,i64,str,str,date,i64,str,bool
2021-10-13,2021-10-13,2,3900,"""VITO""","""2""",2025-10-13,5,"""Grå""",false
2020-01-29,2020-01-29,2,5750,"""CADDY""","""2""",2026-01-29,6,"""Svart (også bl…",false
2023-11-29,2023-11-29,1,2600,"""Mustang Mach-E…","""5""",2027-11-29,6,"""Svart (også bl…",true
2020-01-27,2020-01-27,2,5750,"""CADDY""","""2""",2024-01-27,1,"""Hvit (også ant…",false
2019-01-30,2019-01-30,2,2600,"""F-150 Platinum…","""2""",2024-05-03,6,"""Svart (også bl…",false
2020-05-19,2020-05-19,1,5750,"""CRAFTER""","""2""",2024-05-19,1,"""Hvit (også ant…",false
2020-09-11,2022-02-16,1,5150,"""CITIGO""","""5""",2024-09-11,1,"""Hvit (også ant…",true
2022-05-12,2022-05-12,1,5480,"""YARIS""","""1,5""",2026-05-12,4,"""Grønn (også gr…",false
2022-11-04,2022-11-04,2,5480,"""RAV4""","""1,5,5""",2026-11-04,6,"""Svart (også bl…",false


In [50]:
# 6.
# Skriv hele datasettet til en fil: kjoretoyinfo_preppet.parquet.

pl_df.write_parquet("filer/kjoretoyinfo_preppet.parquet")